In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import re
import os

### 20년도 매출 데이터 확인

In [2]:
file_names = list(os.walk('../datas/huge/project_visualization'))[0][2]
csvs = [pd.read_csv(os.path.join('../datas/huge/project_visualization', file_name), encoding='cp949') for file_name in file_names]
df_sales_2020 = pd.concat(csvs)
df_sales_2019 = pd.concat([
    pd.read_csv('../datas/huge/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='cp949'),
])

In [3]:
df_sales_2020

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO
0,0,2020,1,A,골목상권,1000001,계동길,CS100001,한식음식점,431442455,...,345,8752,7776,611,3828,4356,3428,2614,1690,19
1,1,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161,...,0,592,694,145,368,240,368,96,69,1
2,2,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866,...,241,622,508,0,134,245,275,314,162,3
3,3,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,156728306,...,33,1281,2014,5,1361,1116,331,325,157,3
4,4,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,257814129,...,1205,10426,14742,234,4316,7089,6188,4351,2989,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2020,4,A,골목상권,1000052,녹사평대로32길,CS300024,운동/경기용품,856662,...,0,48,0,0,0,0,48,0,0,3
996,996,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100001,한식음식점,418790754,...,915,5371,5350,245,5771,2713,1175,605,213,7
997,997,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100002,중식음식점,11078526,...,94,274,174,0,210,124,28,86,0,1
998,998,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100003,일식음식점,90936583,...,252,276,73,0,68,58,121,68,35,1


In [4]:
df_sales_2019

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2019,1,A,골목상권,1000985,암사길,CS200003,예술학원,20061598,125,...,0,93,32,0,0,32,93,0,0,3
1,2019,1,A,골목상권,1000267,돌곶이로8길,CS200037,노래방,510636,21,...,8,4,17,0,0,0,9,4,8,1
2,2019,1,R,전통시장,1001445,영도시장,CS300027,섬유제품,4038928,33,...,0,17,0,0,0,17,0,0,0,2
3,2019,1,A,골목상권,1000789,난곡로24길,CS300017,시계및귀금속,5085873,10,...,0,5,5,0,0,0,5,0,5,1
4,2019,1,A,골목상권,1000930,언주로81길,CS200001,일반교습학원,527027,53,...,53,53,0,0,0,0,0,53,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135573,2019,4,R,전통시장,1001469,양재시장,CS300011,일반의류,14282341,48,...,0,0,48,0,0,48,0,0,0,2
135574,2019,4,A,골목상권,1000181,망우로21길,CS100004,양식음식점,562291801,23923,...,228,12935,9718,469,6166,6432,4792,3474,1319,2
135575,2019,4,A,골목상권,1000441,세무서2길,CS300002,편의점,518371562,81848,...,17654,57240,24608,2813,30887,15345,14338,12869,5596,2
135576,2019,4,D,발달상권,1001044,양재역_3,CS200001,일반교습학원,15509601,56,...,6,25,31,0,0,12,44,0,0,6


In [5]:
df_sales_2020.drop_duplicates()

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO
0,0,2020,1,A,골목상권,1000001,계동길,CS100001,한식음식점,431442455,...,345,8752,7776,611,3828,4356,3428,2614,1690,19
1,1,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161,...,0,592,694,145,368,240,368,96,69,1
2,2,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866,...,241,622,508,0,134,245,275,314,162,3
3,3,2020,1,A,골목상권,1000001,계동길,CS100004,양식음식점,156728306,...,33,1281,2014,5,1361,1116,331,325,157,3
4,4,2020,1,A,골목상권,1000001,계동길,CS100005,제과점,257814129,...,1205,10426,14742,234,4316,7089,6188,4351,2989,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2020,4,A,골목상권,1000052,녹사평대로32길,CS300024,운동/경기용품,856662,...,0,48,0,0,0,0,48,0,0,3
996,996,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100001,한식음식점,418790754,...,915,5371,5350,245,5771,2713,1175,605,213,7
997,997,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100002,중식음식점,11078526,...,94,274,174,0,210,124,28,86,0,1
998,998,2020,4,A,골목상권,1000053,녹사평대로40나길,CS100003,일식음식점,90936583,...,252,276,73,0,68,58,121,68,35,1


In [6]:
df_sales_2020[df_sales_2020['TRDAR_SE_CD_NM'] == '전통시장']

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO
19,19,2020,2,R,전통시장,1001352,번동북부골목시장,CS200028,미용실,90507273,...,75,1862,1525,28,382,472,1082,983,440,8
20,20,2020,2,R,전통시장,1001352,번동북부골목시장,CS200029,네일숍,521733,...,0,5,10,0,5,5,0,5,0,0
95,95,2020,2,R,전통시장,1001352,번동북부골목시장,CS200031,세탁소,10754472,...,43,298,256,21,86,192,106,149,0,2
117,117,2020,4,R,전통시장,1001460,관악신사시장,CS300018,의약품,271160578,...,0,10719,13341,197,2771,4362,5964,5889,4877,4
463,463,2020,4,R,전통시장,1001475,강남개포시장,CS100007,치킨전문점,87505216,...,1098,1926,1308,195,438,348,937,1027,291,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,2020,3,R,전통시장,1001490,길동복조리시장,CS300022,화장품,1010676465,...,118,15494,28948,0,1454,4155,4614,14528,19690,6
887,887,2020,3,R,전통시장,1001490,길동복조리시장,CS300027,섬유제품,170000001,...,25,934,2081,0,192,279,562,805,1177,5
888,888,2020,3,R,전통시장,1001490,길동복조리시장,CS300028,화초,60772803,...,245,2104,665,35,1474,735,280,210,35,4
889,889,2020,3,R,전통시장,1001490,길동복조리시장,CS300029,애완동물,27813477,...,0,379,1105,0,180,174,321,494,315,2


In [7]:
df_sales_2019[df_sales_2019['상권_구분_코드_명'] == '전통시장']

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2019,1,R,전통시장,1001445,영도시장,CS300027,섬유제품,4038928,33,...,0,17,0,0,0,17,0,0,0,2
6,2019,1,R,전통시장,1001483,암사종합시장,CS300008,수산물판매,1712543938,108896,...,511,40273,68502,116,2309,12234,21650,33132,39332,9
14,2019,1,R,전통시장,1001316,서울약령시장,CS200008,한의원,3329153658,38642,...,26,14251,24366,10,1441,3620,6465,7383,19683,82
15,2019,1,R,전통시장,1001314,경동시장,CS300007,육류판매,4587060783,84286,...,5,31843,51739,30,1370,6433,11198,20193,44356,30
28,2019,1,R,전통시장,1001323,답십리시장,CS200028,미용실,100328154,2550,...,0,1470,1080,50,483,623,596,604,194,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135547,2019,4,R,전통시장,1001317,청량리농수산물시장,CS300009,청과상,86437896,1553,...,0,1553,0,0,0,518,517,0,518,15
135565,2019,4,R,전통시장,1001416,공항시장,CS100010,커피-음료,15675910,2016,...,182,950,907,15,777,392,358,262,54,1
135566,2019,4,R,전통시장,1001471,영동전통시장,CS300010,반찬가게,4773539895,118316,...,1871,41841,74436,661,11780,25687,27632,27036,23478,12
135571,2019,4,R,전통시장,1001427,현대시장,CS300019,의료기기,32120476,757,...,0,344,413,0,0,48,122,233,354,3


In [8]:
df_sales_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132321 entries, 0 to 999
Data columns (total 81 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Unnamed: 0                 132321 non-null  int64 
 1   STDR_YY_CD                 132321 non-null  int64 
 2   STDR_QU_CD                 132321 non-null  int64 
 3   TRDAR_SE_CD                132321 non-null  object
 4   TRDAR_SE_CD_NM             132321 non-null  object
 5   TRDAR_CD                   132321 non-null  int64 
 6   TRDAR_CD_NM                132321 non-null  object
 7   SVC_INDUTY_CD              132321 non-null  object
 8   SVC_INDUTY_CD_NM           132321 non-null  object
 9   THSMON_SELNG_AMT           132321 non-null  int64 
 10  THSMON_SELNG_CO            132321 non-null  int64 
 11  MDWK_SELNG_RATE            132321 non-null  int64 
 12  WKEND_SELNG_RATE           132321 non-null  int64 
 13  MON_SELNG_RATE             132321 non-null  int

In [9]:
df_sales_2020[df_sales_2020['TRDAR_CD_NM'].str.contains('방림')]

,Unnamed: 0,STDR_YY_CD,STDR_QU_CD,TRDAR_SE_CD,TRDAR_SE_CD_NM,TRDAR_CD,TRDAR_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,...,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO,STOR_CO


In [10]:
df_sales_2020 = df_sales_2020.drop('Unnamed: 0', axis=1)

In [11]:
df_sales_2020.columns = df_sales_2019.columns

In [12]:
df_sales_2020.to_csv('../datas/huge/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2020.csv', encoding='cp949')

### 시장주소, 생필품 데이터 확인

- 창동신창시장 -> 신창시장으로 통일
- 송화시장 -> 송화골목시장으로 통일
- 공릉동 도깨비시장 -> 공릉동도깨비시장	
- 시장주소에 있는 대림시장은 삭제

In [30]:
####################################
market_mapping = {
    '창동신창시장': '신창시장',
    '송화시장': '송화골목시장',
    '공릉동 도깨비시장': '공릉동도깨비시장',
}
# df_market_address에서 market_mapping을 적용해 replace한 다음, 그대로 데이터 저장

In [14]:
####################################
df_customer = pd.read_csv('../datas/huge/df_customer.csv', encoding='cp949')
df_market_address = pd.read_csv('../datas/huge/df_market_address.csv', encoding='cp949')
df_market_address2 = pd.read_csv('../datas/huge/df_market_address2.csv', encoding='cp949')
df_necessity = pd.read_csv('../datas/huge/df_necessity.csv', encoding='cp949')
df_necessity2 = pd.read_csv('../datas/huge/df_necessity2.csv', encoding='cp949')

In [15]:
# df_market_address = df_market_address.drop(['Unnamed: 0', '주소명', '자치구'], axis=1)

In [16]:
# df_market_address2 = df_market_address2.drop('Unnamed: 0', axis=1)

In [99]:
# pd.concat([df_market_address, df_market_address2]).drop_duplicates(keep=False)

,시장이름,위도,경도
20,공릉동 도깨비시장,37.622888,127.077240
163,송화시장,37.548727,126.835716
292,신창시장,37.640302,127.038832
20,공릉동도깨비시장,37.622888,127.077240
59,대림시장,37.491372,126.907663
164,송화골목시장,37.548727,126.835716
293,창동신창시장,37.640302,127.038832


In [18]:
# df_market_address[df_market_address['시장이름']=='대림시장']

,시장이름,위도,경도
58,대림시장,37.586763,126.91761


In [19]:
# df_market_address['시장이름'].unique()

array(['갈현시장', '강남시장', '강남역 지하도상가', '강남터미널지하도상점가1구역', '강남터미널지하도상점가2구역',
       '강남터미널지하도상점가3구역', '강북종합골목시장', '강북종합시장', '개봉중앙시장', '개봉프라자',
       '개포2동 중심상가', '경동광성상가', '경동시장', '경창시장', '고덕전통시장', '고속터미널 지하철역 상점가 ',
       '고척근린시장', '고척프라자', '공덕시장', '공릉동 도깨비시장', '공무원연금매장 상록스토아 개포점',
       '공항시장', '관악신사시장(신림4동)', '관악종합시장', '광성시장', '광장골목시장', '광장시장', '광희시장',
       '구로시장', '구의시장', '굽은다리골목시장', '금남시장', '금천교시장', '길동골목시장', '길음시장',
       '까치산시장', '깡통시장(제기상가번영회)', '낙성대 시장(봉천7동)', '낙원상가', '낙원지하시장(대일상가)',
       '남구로시장', '남대문로 지하상가', '남대문시장', '남문시장', '남부골목시장', '남부종합시장',
       '남부화곡시장', '남서울상가', '남평화시장', '노룬산골목시장', '노룬산시장', '녹번시장', '논현종합시장',
       '능동로골목시장', '답십리시장', '답십리현대시장', '대림골목시장', '대림시장', '대림중앙시장', '대명시장',
       '대명시장 및 주변상점가', '대신시장', '대원종합시장', '대조시장', '도림시장', '도선동상점가',
       '도화동 상점가', '돈암제일시장', '동남상가', '동대문문구완구종합시장', '동대문상가A동', '동대문상가B동',
       '동대문상가C동', '동대문종합시장', '동대문종합시장D동상가', '동대문지하쇼핑센터', '동문시장', '동부골목시장',
       '동부시장', '동부청과시장', '동북시장', '동서시장', '동서울시장', '동성타워프라자', '동소문 스카이상가',

In [20]:
# df_market_address2['시장이름'].unique()

array(['갈현시장', '강남시장', '강남역 지하도상가', '강남터미널지하도상점가1구역', '강남터미널지하도상점가2구역',
       '강남터미널지하도상점가3구역', '강북종합골목시장', '강북종합시장', '개봉중앙시장', '개봉프라자',
       '개포2동 중심상가', '경동광성상가', '경동시장', '경창시장', '고덕전통시장', '고속터미널 지하철역 상점가 ',
       '고척근린시장', '고척프라자', '공덕시장', '공릉동도깨비시장', '공무원연금매장 상록스토아 개포점', '공항시장',
       '관악신사시장(신림4동)', '관악종합시장', '광성시장', '광장골목시장', '광장시장', '광희시장', '구로시장',
       '구의시장', '굽은다리골목시장', '금남시장', '금천교시장', '길동골목시장', '길음시장', '까치산시장',
       '깡통시장(제기상가번영회)', '낙성대 시장(봉천7동)', '낙원상가', '낙원지하시장(대일상가)', '남구로시장',
       '남대문로 지하상가', '남대문시장', '남문시장', '남부골목시장', '남부종합시장', '남부화곡시장',
       '남서울상가', '남평화시장', '노룬산골목시장', '노룬산시장', '녹번시장', '논현종합시장', '능동로골목시장',
       '답십리시장', '답십리현대시장', '대림골목시장', '대림시장', '대림중앙시장', '대명시장',
       '대명시장 및 주변상점가', '대신시장', '대원종합시장', '대조시장', '도림시장', '도선동상점가',
       '도화동 상점가', '돈암제일시장', '동남상가', '동대문문구완구종합시장', '동대문상가A동', '동대문상가B동',
       '동대문상가C동', '동대문종합시장', '동대문종합시장D동상가', '동대문지하쇼핑센터', '동문시장', '동부골목시장',
       '동부시장', '동부청과시장', '동북시장', '동서시장', '동서울시장', '동성타워프라자', '동소문 스카이상가',


- 일련번호 drop
- 년도, 분기 컬럼 분리
- 품목 이름 -> 품목이름
- 자치구 -> 자치구이름

In [21]:
# 생필품 데이터에서 년도-분기 매칭 완료
condlist = []
choicelist_year = []
choicelist_quarter = []

for year_month in df_necessity['년도-월'].unique():
    year, month = year_month.split('-')
    quarter = ''
    if month in {'01', '02', '03'}:
        quarter = 1
    elif month in {'04', '05', '06'}:
        quarter = 2
    elif month in {'07', '08', '09'}:
        quarter = 3
    elif month in {'10', '11', '12'}:
        quarter = 4
    condlist.append(df_necessity['년도-월'] == year_month)
    choicelist_year.append(year)
    choicelist_quarter.append(quarter)

df_necessity['년도'] = np.select(condlist, choicelist_year)
df_necessity['분기'] = np.select(condlist, choicelist_quarter)

In [22]:
df_necessity['년도'] = df_necessity['년도'].astype('int64')
df_necessity['분기'] = df_necessity['분기'].astype('int64')

In [23]:
df_necessity = df_necessity.drop(['Unnamed: 0', '비고', '업종', '년도-분기'], axis=1).rename(columns={'품목 이름': '품목이름', '자치구': '자치구이름'})

In [24]:
df_necessity2 = df_necessity2.drop(['Unnamed: 0'], axis=1).rename(columns={'품목 이름': '품목이름', '자치구명': '자치구이름'})

In [32]:
df_necessity['시장이름'].unique()

array(['통인시장', '방배종합시장', '방학동도깨비시장', '장위골목시장', '신창시장', '마포농수산물시장', '동원시장',
       '광장시장', '남성시장', '금남시장', '뚝도시장', '후암시장', '청량리종합시장', '남문시장', '남구로시장',
       '신영시장', '고척근린시장', '용문시장', '대림시장', '영등포전통시장', '망원시장', '영천시장',
       '인왕시장', '신원시장(신림1동)', '자양골목시장', '수유재래시장', '경동시장', '우림골목시장', '송화시장',
       '대림중앙시장', '목3동시장', '공릉동 도깨비시장', '돈암제일시장', '원당종합시장', '남대문시장',
       '청담삼익시장', '현대시장', '둔촌역전통시장', '서울중앙시장', '노룬산골목시장', '화곡본동시장', '방이시장',
       '대조시장', '도곡시장', '마천중앙시장', '관악신사시장(신림4동)', '암사종합시장', '상계중앙시장',
       '숭인시장', '방림시장', '구로시장', '영동전통시장'], dtype=object)

In [40]:
df_necessity['시장이름'] = df_necessity['시장이름'].replace(market_mapping)

In [45]:
df_necessity2['시장이름'] = df_necessity2['시장이름'].replace(market_mapping)

### df_necessity, df_necessity2 순
- 우림골목시장 <- 우림시장
- 도곡시장 살림
- 방림시장 죽임
- 남구로시장 현용 데이터만 삭제 (번호-시장이름 오류를 고친 것 때문에 나오는 것일 듯)
- 신영시장, 신영시장
- 목3동시장, 목3동시장

In [74]:
market_mapping2 = {
    '방배종합시장': "남부종합시장", 
    '남성시장': "사당시장", 
    '영동시장': "영동전통시장", 
    '장위골목시장': "장위전통시장", 
    '용문시장': "용문전통시장",
    '신원시장(신림1동)': "신원시장", 
    '수유재래시장': "수유전통시장", 
    '원당종합시장': "인헌시장",
    '마천중앙시장': "마천시장", 
    '관악신사시장(신림4동)': "관악신사시장",
    '우림시장': '우림골목시장',
}
# necessity_dead = ['방림시장']
# necessity2_dead = ['남구로시장']

# test_dead = ['도곡시장', '구로시장']

In [75]:
df_necessity['시장이름'] = df_necessity['시장이름'].replace(market_mapping2)

In [76]:
df_necessity2['시장이름'] = df_necessity2['시장이름'].replace(market_mapping2)

In [95]:

df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='방림시장'].index)
df_necessity2 = df_necessity2.drop(df_necessity2[df_necessity2['시장이름']=='남구로시장'].index)

df_necessity2 = df_necessity2.dropna(subset=['품목이름'])

# for duplicate check(나중에 살려야하는 자료)
# df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='구로시장'].index)
# df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='남구로시장'].index)
# df_necessity2 = df_necessity2.drop(df_necessity2[df_necessity2['시장이름']=='구로시장'].index)
# df_necessity = df_necessity.drop(df_necessity[df_necessity['시장이름']=='도곡시장'].index)

In [96]:
pd.concat([df_necessity, df_necessity2]).drop_duplicates(keep=False)['시장이름'].unique()

array([], dtype=object)

In [97]:
pd.concat([df_necessity, df_necessity2]).drop_duplicates(keep=False)

,일련번호,시장이름,품목이름,실판매규격,가격(원),년도-월,자치구이름,년도,분기


In [98]:
####################################### 저장해야할 데이터
pd.concat([df_necessity, df_necessity2]).drop_duplicates()

,일련번호,시장이름,품목이름,실판매규격,가격(원),년도-월,자치구이름,년도,분기
0,1621162,통인시장,양파(1.5kg망),1kg,2000,2020-12,종로구,2020,4
1,1628949,남부종합시장,호박,1개268g,1500,2020-12,서초구,2020,4
2,1622882,통인시장,양파(1.5kg망),1kg,2000,2020-12,종로구,2020,4
3,1622883,통인시장,상추(100g),400g,3000,2020-12,종로구,2020,4
4,1622885,통인시장,호박(인큐베이터),1개,1000,2020-12,종로구,2020,4
...,...,...,...,...,...,...,...,...,...
171034,406492,방이시장,달걀(특란),10개,2150,2013-01,송파구,2013,1
171035,406493,방이시장,"조기(냉동,국산)",1마리(20cm),1780,2013-01,송파구,2013,1
171036,406494,방이시장,"명태(러시아,냉동)",1마리(45cm),5000,2013-01,송파구,2013,1
171037,406495,방이시장,"오징어(냉동,국산)",1마리(25cm),3000,2013-01,송파구,2013,1


In [27]:
df_necessity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171039 entries, 0 to 171038
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   일련번호    171039 non-null  int64 
 1   시장이름    171039 non-null  object
 2   품목이름    171039 non-null  object
 3   실판매규격   171039 non-null  object
 4   가격(원)   171039 non-null  int64 
 5   년도-월    171039 non-null  object
 6   자치구이름   171039 non-null  object
 7   년도      171039 non-null  int64 
 8   분기      171039 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 11.7+ MB


In [28]:
df_necessity2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164457 entries, 0 to 164456
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   년도      164457 non-null  int64 
 1   분기      164457 non-null  int64 
 2   일련번호    164457 non-null  int64 
 3   년도-월    164457 non-null  object
 4   시장이름    164457 non-null  object
 5   품목이름    164453 non-null  object
 6   실판매규격   164457 non-null  object
 7   가격(원)   164457 non-null  int64 
 8   자치구이름   164457 non-null  object
dtypes: int64(4), object(5)
memory usage: 11.3+ MB


In [29]:
df_necessity.groupby(['시장이름', '품목이름', '실판매규격']).count()

일련번호  가격(원)  년도-월  자치구이름  년도  분기
시장이름 품목이름       실판매규격                                       
경동시장 고등어(냉동,국산) 1마리(20cm)      2      2     2      2   2   2
     고등어(생물,국산) 1ㅁ리(25cm)      1      1     1      1   1   1
                1마리((25cm)     1      1     1      1   1   1
                1마리(20cm)      5      5     5      5   5   5
                1마리(258g)      1      1     1      1   1   1
...                          ...    ...   ...    ...  ..  ..
후암시장 조기(생물,수입산) 1마리 35cm       1      1     1      1   1   1
                1마리20cm)       6      6     6      6   6   6
     조기(중국산,생물) 1마리20          4      4     4      4   4   4
                1마리25cm)       3      3     3      3   3   3
     호박         1개            23     23    23     23  23  23

[9306 rows x 6 columns]